#### Imports

La idea de esta variante, es predecir solamente los productos que tengan mas de 12 meses de datos (usando los ultimos 12 solamente), y aquellos con menos datos, predecir la media

In [212]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from skopt import BayesSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.metrics import MeanSquaredError
from keras.regularizers import l2
from keras.layers import BatchNormalization
from skopt.space import Categorical
from sklearn.metrics import mean_squared_error
import json

In [213]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [214]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


#### Pre procesamiento

In [215]:
final_dataset['periodo'] = pd.to_datetime(final_dataset['periodo'], format='%Y%m')


In [216]:
# Promediamos las vtas de agosto, como las de julio y setpiembre
july_september_data = final_dataset[
    final_dataset['periodo'].isin([pd.Timestamp('2019-07-01'), pd.Timestamp('2019-09-01')])
]

# Agrupar por producto y calcular el promedio
august_average = july_september_data.groupby('product_id')['y'].mean().reset_index()
august_average['periodo'] = pd.Timestamp('2019-08-01')

# Reemplazar los valores de agosto en el DataFrame original
final_dataset = final_dataset.set_index(['product_id', 'periodo'])
final_dataset.update(august_average.set_index(['product_id', 'periodo']))
final_dataset = final_dataset.reset_index()

In [217]:
# Agreog el MES del quarter, en vez de si es close_quarter
final_dataset['month'] = final_dataset['periodo'].dt.month

def month_in_quarter(month):
    if month in [1, 4, 7, 10]:
        return 1  # Primer mes del trimestre
    elif month in [2, 5, 8, 11]:
        return 2  # Segundo mes del trimestre
    elif month in [3, 6, 9, 12]:
        return 3  # Tercer mes del trimestre

final_dataset['month_in_quarter'] = final_dataset['month'].apply(month_in_quarter)

# Remover la columna 'month' si no es necesaria
final_dataset = final_dataset.drop(columns=['month'])

In [218]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'month_in_quarter', 'brand', 'sku_size', 'cat1', 'cat2', 'cat3','y'] #'close_quarter'
non_scalable_columns = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'plan_precios_cuidados', 'month_in_quarter'] # 'close_quarter'
n_features = len(columns)

Valido el promedio de agosto (este es el mes en que la empresa decidio no vender por unos dias)

In [219]:
# Filtrar los datos para los productos en julio, agosto y septiembre de 2020
august_data = final_dataset[
    final_dataset['periodo'] == pd.Timestamp('2019-08-01')
]

july_data = final_dataset[
    final_dataset['periodo'] == pd.Timestamp('2019-07-01')
]

september_data = final_dataset[
    final_dataset['periodo'] == pd.Timestamp('2019-09-01')
]

# Mostrar algunos datos de agosto para revisar que el promedio se haya aplicado
print("Datos de agosto de 2019:")
display(august_data.head())

# Mostrar algunos datos de julio para comparar
print("\nDatos de julio de 2019:")
display(july_data.head())

# Mostrar algunos datos de septiembre para comparar
print("\nDatos de septiembre de 2019:")
display(september_data.head())

# Verificar el cálculo del promedio manualmente para algunos productos
product_ids = august_data['product_id'].unique()

for product_id in product_ids[:5]:  # Limitar a los primeros 5 productos para revisar
    july_sales = july_data[july_data['product_id'] == product_id]['y'].values
    september_sales = september_data[september_data['product_id'] == product_id]['y'].values
    august_sales = august_data[august_data['product_id'] == product_id]['y'].values

    print(f"\nProducto ID: {product_id}")
    print(f"Ventas de julio: {july_sales}")
    print(f"Ventas de septiembre: {september_sales}")
    print(f"Ventas de agosto (calculadas): {august_sales}")
    if len(july_sales) > 0 and len(september_sales) > 0:
        calculated_average = (july_sales[0] + september_sales[0]) / 2
        print(f"Promedio calculado: {calculated_average}")


Datos de agosto de 2019:


,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age,month_in_quarter
31,20001,2019-08-01,0,365,1263.13762,1669.499395,HC,ROPA LAVADO,Liquido,ARIEL,3000,262.82367,0,31,2
67,20002,2019-08-01,0,315,827.28604,1078.318850,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,531.76312,0,31,2
103,20003,2019-08-01,0,306,655.51370,841.487150,FOODS,ADEREZOS,Mayonesa,NATURA,475,1097.55623,0,31,2
139,20004,2019-08-01,0,304,508.31433,653.943295,FOODS,ADEREZOS,Mayonesa,NATURA,240,1284.38214,0,31,2
175,20005,2019-08-01,0,264,567.01790,812.638930,FOODS,ADEREZOS,Mayonesa,NATURA,120,751.19612,0,31,2



Datos de julio de 2019:


,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age,month_in_quarter
30,20001,2019-07-01,0,432,1782.81423,1678.99318,HC,ROPA LAVADO,Liquido,ARIEL,3000,107.73060,0,30,1
66,20002,2019-07-01,0,401,1077.79994,1066.44999,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,480.50413,0,30,1
102,20003,2019-07-01,0,380,746.64283,715.20314,FOODS,ADEREZOS,Mayonesa,NATURA,475,1037.85386,0,30,1
138,20004,2019-07-01,0,359,536.82933,521.71519,FOODS,ADEREZOS,Mayonesa,NATURA,240,899.12650,0,30,1
174,20005,2019-07-01,0,372,801.30821,745.74978,FOODS,ADEREZOS,Mayonesa,NATURA,120,454.10572,0,30,1



Datos de septiembre de 2019:


,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age,month_in_quarter
32,20001,2019-09-01,0,392,1734.24194,1660.00561,HC,ROPA LAVADO,Liquido,ARIEL,3000,32.67653,1,32,3
68,20002,2019-09-01,0,438,1095.09822,1090.18771,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,8.78311,1,32,3
104,20003,2019-09-01,0,470,977.10777,967.77116,FOODS,ADEREZOS,Mayonesa,NATURA,475,642.31918,1,32,3
140,20004,2019-09-01,0,466,818.26309,786.17140,FOODS,ADEREZOS,Mayonesa,NATURA,240,1562.02448,1,32,3
176,20005,2019-09-01,0,454,932.68979,879.52808,FOODS,ADEREZOS,Mayonesa,NATURA,120,1057.38804,1,32,3



Producto ID: 20001
Ventas de julio: [1678.99318]
Ventas de septiembre: [1660.00561]
Ventas de agosto (calculadas): [1669.499395]
Promedio calculado: 1669.4993949999998

Producto ID: 20002
Ventas de julio: [1066.44999]
Ventas de septiembre: [1090.18771]
Ventas de agosto (calculadas): [1078.31885]
Promedio calculado: 1078.31885

Producto ID: 20003
Ventas de julio: [715.20314]
Ventas de septiembre: [967.77116]
Ventas de agosto (calculadas): [841.48715]
Promedio calculado: 841.4871499999999

Producto ID: 20004
Ventas de julio: [521.71519]
Ventas de septiembre: [786.1714]
Ventas de agosto (calculadas): [653.943295]
Promedio calculado: 653.943295

Producto ID: 20005
Ventas de julio: [745.74978]
Ventas de septiembre: [879.52808]
Ventas de agosto (calculadas): [812.63893]
Promedio calculado: 812.6389300000001


Muetro cual es el primer mes de venta de cada producto, para debug

In [220]:
first_months = final_dataset.groupby('product_id')['periodo'].min().reset_index()
first_months.columns = ['product_id', 'first_month']

# Mostrar los resultados
print("Primer mes de cada producto:")
display(first_months.sort_values(by=['first_month', 'product_id']).tail(150))

Primer mes de cada producto:


,product_id,first_month
518,20700,2018-10-01
525,20709,2018-10-01
632,20917,2018-10-01
669,20990,2018-10-01
675,21001,2018-10-01
...,...,...
657,20962,2019-09-01
663,20975,2019-09-01
672,20995,2019-09-01
711,21087,2019-09-01


In [221]:
categorical_columns = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size']

# Transformar las columnas categóricas en numéricas
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    final_dataset[col] = le.fit_transform(final_dataset[col])
    label_encoders[col] = le

In [222]:
grouped = final_dataset.groupby(['product_id', 'cat1', 'cat2', 'cat3'])

for name, group in grouped:
    print(f"Group name: {name}")
    print(group.head())
    print("\n")
    # break 

Group name: (20001, 1, 10, 47)
   product_id    periodo  plan_precios_cuidados  cust_request_qty  \
0       20001 2017-01-01                      0               479   
1       20001 2017-02-01                      0               432   
2       20001 2017-03-01                      0               509   
3       20001 2017-04-01                      0               279   
4       20001 2017-05-01                      0               701   

   cust_request_tn           y  cat1  cat2  cat3  brand  sku_size  \
0        937.72717   934.77222     1    10    47      0        63   
1        833.72187   798.01620     1    10    47      0        63   
2       1330.74697  1303.35771     1    10    47      0        63   
3       1132.94430  1069.96130     1    10    47      0        63   
4       1550.68936  1502.20132     1    10    47      0        63   

   stock_final  close_quarter  age  month_in_quarter  
0          NaN              0    0                 1  
1          NaN              0

In [223]:
scalers = {col: {} for col in columns}
scaled_data_list = []

for (product_id, _, _, _ ), group in grouped:
    scaled_group = group.copy()
    for col in columns:
        if col not in non_scalable_columns:
            scaler = StandardScaler()
            # print(col)
            scaled_group[col] = scaler.fit_transform(group[[col]])
            scalers[col][product_id] = scaler
    scaled_data_list.append(scaled_group)

# Combinar todos los datos escalados en un solo DataFrame
scaled_data = pd.concat(scaled_data_list)

In [224]:
display(scaled_data[columns])

,plan_precios_cuidados,cust_request_qty,cust_request_tn,month_in_quarter,brand,sku_size,cat1,cat2,cat3,y
0,0,0.461891,-1.704215,1,0,63,1,10,47,-1.602615
1,0,-0.046911,-2.039835,2,0,63,1,10,47,-2.064107
2,0,0.786658,-0.435961,3,0,63,1,10,47,-0.358798
3,0,-1.703222,-1.074260,1,0,63,1,10,47,-1.146410
4,0,2.865166,0.273781,2,0,63,1,10,47,0.312213
...,...,...,...,...,...,...,...,...,...,...
21615,0,-0.652507,-0.792584,2,23,28,2,6,18,-0.837906
21616,0,-0.268679,-0.649760,3,23,28,2,6,18,-0.641461
21617,0,0.115148,-0.595870,1,23,28,2,6,18,-0.587808
21618,0,-0.012794,-0.290890,2,23,28,2,6,18,-0.284178


In [225]:
# scaled_data[scaled_data['cat1'] != 0]

#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos hasta N-2 e Y va a ser N (siendo N la cantidad de ventas para ese producto)

In [226]:
timeframe = 18

In [227]:
def prepare_data(data,validation=False):
    X, y, ids = [], [], []
    unique_product_ids = data['product_id'].unique()
    predicciones = {}

    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        if len(product_values) < timeframe + 2:
            if validation:
                predicciones[product_id] = np.mean(product_values)
            continue
        
        
        if validation:
            X.append(product_values[-(timeframe + 2):-2])
            y.append(product_values[-1, -1]) 
            ids.append(product_id)
        else:
            # Todas las observaciones menos las últimas dos. Como despues necesito agregar esos 2 registros
            # para predecir el future, agrego dos registros previos
            X.append(product_values[-(timeframe + 2):-2])
            y.append(product_values[-1, -1])      # Última observación
    
    if validation:
        return np.array(X), np.array(y), predicciones, ids
    else:
        return np.array(X), np.array(y), predicciones, []


Definimos la funcion para crear el modelo LSTM, sobre este se ejecutara la optimizacion bayesiana

In [228]:
predictions = []

In [229]:
data = scaled_data[columns].values
X, y, _, _ = prepare_data(scaled_data)

In [230]:
display(X.shape)
display(y.shape)

(599, 18, 10)

(599,)

#### Revisamos que la funcion genere registros coherentes

In [231]:
# Muestro todos los registros del ultimo producto
unique_product_ids = scaled_data['product_id'].unique()
product_data = scaled_data[scaled_data['product_id'] == unique_product_ids[0]] #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft
product_values = product_data[columns].values
display(product_values)

array([[ 0.00000000e+00,  4.61890740e-01, -1.70421544e+00,
         1.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
        -1.60261492e+00],
       [ 0.00000000e+00, -4.69107783e-02, -2.03983510e+00,
         2.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
        -2.06410735e+00],
       [ 0.00000000e+00,  7.86657667e-01, -4.35961139e-01,
         3.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
        -3.58798283e-01],
       [ 0.00000000e+00, -1.70322210e+00, -1.07425999e+00,
         1.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
        -1.14641024e+00],
       [ 0.00000000e+00,  2.86516600e+00,  2.73781431e-01,
         2.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
         3.

In [232]:
display(X[0]) #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft

array([[ 0.00000000e+00,  2.12902763e-01, -5.23494287e-01,
         2.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
        -3.90721119e-01],
       [ 0.00000000e+00,  6.56750896e-01, -8.62710503e-01,
         3.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
        -8.73642626e-01],
       [ 0.00000000e+00,  2.34553892e-01,  2.02480217e-01,
         1.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
         2.04931260e-01],
       [ 0.00000000e+00,  1.06812234e+00,  1.27153863e+00,
         2.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
         1.32039993e+00],
       [ 0.00000000e+00, -3.82503269e-01,  5.27709100e-01,
         3.00000000e+00,  0.00000000e+00,  6.30000000e+01,
         1.00000000e+00,  1.00000000e+01,  4.70000000e+01,
         9.

In [233]:
display(y[0]) #<= cambiar el indice entre 0 y -1

0.3206062727520878

In [234]:
display(len(X[0]))

18

#### Entrenamiento del modelo

#### Armado de los modelos

#### Guardo los mejores hiperparametros

In [235]:
import numpy as np
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split

# TODO: Ver que hace esta funcion, tiene sentido?
def aggregate_features(X):
    X_agg = np.zeros((X.shape[0], X.shape[1], 14))  # (617, 14, 14)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X_agg[i, j, 0] = np.mean(X[i, j, :])
            X_agg[i, j, 1] = np.std(X[i, j, :])
            X_agg[i, j, 2] = np.max(X[i, j, :])

    return X_agg


X_agg = aggregate_features(X)

print("Shape of X_agg:", X_agg.shape)

X_train, X_test, y_train, y_test = train_test_split(X_agg.reshape(X_agg.shape[0], -1), y, test_size=0.1, random_state=42)


tpot = TPOTRegressor(
    generations=100,
    population_size=100,
    random_state=42,
    verbosity=2,
    mutation_rate=0.6,
    scoring='neg_mean_squared_error',
    cv=10,
    n_jobs=-1,
    periodic_checkpoint_folder="./tpot_results",
    # early_stop=5
)

tpot.fit(X_train, y_train)

print(tpot.score(X_test, y_test))

tpot.export('tpot_best_model.py')


Shape of X_agg: (599, 18, 14)
Optimization Progress:   0%|          | 0/2550 [00:00<?, ?pipeline/s]

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.1

                                                                                
Generation 1 - Current best internal CV score: -0.5570712910712372
                                                                               
Generation 2 - Current best internal CV score: -0.5570712910712372
                                                                               
Generation 3 - Current best internal CV score: -0.5570712910712372
                                                                               
Generation 4 - Current best internal CV score: -0.5570712910712372
                                                                                 
Generation 5 - Current best internal CV score: -0.5557291540256307
                                                                                 
Generation 6 - Current best internal CV score: -0.5549850085787023
                                                                               
Generation 7 - Current best inter

In [236]:
X_f, product_ids, predictions, ids = prepare_data(scaled_data, validation=True)

In [237]:
X_f_agg = aggregate_features(X_f)

scaled_predictions = tpot.predict(X_f_agg.reshape(X_f_agg.shape[0], -1))

In [238]:
scaled_predictions_1d =  scaled_predictions.reshape(-1)
# display(scaled_predictions_1d)

In [239]:
predictions_df = pd.DataFrame({
    'product_id': ids,
    'predicted_y': scaled_predictions_1d
})

# Iterar sobre cada fila del DataFrame predictions_df
for index, row in predictions_df.iterrows():
    # Obtener el ID del producto y la predicción escalada para este producto
    product_id = row['product_id']
    scaled_prediction = row['predicted_y']
    
    # Obtener el escalador correspondiente a 'predicted_y' para este producto
    scaler = scalers['y'][(product_id)]
    
    # Aplicar la inversa del escalador a la predicción 'predicted_y' para este producto
    inverse_scaled_prediction = scaler.inverse_transform([[scaled_prediction]])[0][0]
    
    # Reemplazar la predicción escalada con la predicción invertida en el DataFrame final
    predictions_df.at[index, 'predicted_y'] = inverse_scaled_prediction

In [240]:
prediction_list = []
for product_id, pronostico in predictions.items():
    prediction_list.append({'product_id': product_id, 'predicted_y': pronostico})

# Convert the list of dictionaries to a DataFrame
predictions_df = pd.concat([predictions_df, pd.DataFrame(prediction_list)], ignore_index=True)

In [241]:
display(predictions_df)

,product_id,predicted_y
0,20001,1199.070401
1,20002,874.359454
2,20003,677.580683
3,20004,522.502438
4,20005,616.808874
...,...,...
775,21263,14.300000
776,21265,5.310000
777,21266,5.310000
778,21267,9.010000


In [242]:
december_2019_data = final_dataset[final_dataset['periodo'] == pd.Timestamp('2019-12-01')]
total_sales_december_2019 = december_2019_data.groupby('product_id')['y'].sum().reset_index()
total_sales_december_2019.columns = ['product_id', 'total_sales_201912']
display(total_sales_december_2019)

,product_id,total_sales_201912
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443
...,...,...
775,21263,0.01270
776,21265,0.05007
777,21266,0.05121
778,21267,0.01569


In [243]:
total_sales_december_2019 = total_sales_december_2019.sort_values(by='product_id')
predictions_df = predictions_df.sort_values(by='product_id')


In [244]:
# Calcular la resta producto a producto y tomar el valor absoluto
multiplicator = 1
difference_sum = np.sum(np.abs(total_sales_december_2019['total_sales_201912'].values - predictions_df['predicted_y'].values * multiplicator))

# Calcular la suma total de todas las ventas en total_sales_december_2019
total_sales_sum = total_sales_december_2019['total_sales_201912'].sum()
metric_value = difference_sum / total_sales_sum
print(f"Valor de la métrica: {metric_value}")


Valor de la métrica: 0.27335902126917627


Calculo el multiplicador optimo para cada producto

armo un cluster de los multiplicadores

#### Predicciones a feb

In [245]:
def prepare_data(data):
    X, y = [], []
    unique_product_ids = data['product_id'].unique()
    predicciones = {}

    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        if len(product_values) < timeframe + 2:
            predicciones[product_id] = np.mean(product_values)
            continue
        
        
        X.append(product_values[-timeframe:])
        y.append(product_id)
    
    return np.array(X), np.array(y), predicciones


In [246]:
X_kaggle, product_ids_kaggle, predictions_kaggle = prepare_data(scaled_data)

In [247]:
X_kaggle = aggregate_features(X_kaggle)
scaled_predictions_kaggle = tpot.predict(X_f_agg.reshape(X_f_agg.shape[0], -1))

In [248]:
scaled_predictions_1d_kaggle =  scaled_predictions_kaggle.reshape(-1)
# display(scaled_predictions_1d_kaggle)

In [249]:
predictions_df_kaggle = pd.DataFrame({
    'product_id': product_ids_kaggle,
    'predicted_y': scaled_predictions_1d_kaggle
})

# Iterar sobre cada fila del DataFrame predictions_df
for index, row in predictions_df_kaggle.iterrows():
    # Obtener el ID del producto y la predicción escalada para este producto
    product_id = row['product_id']
    scaled_prediction = row['predicted_y']
    
    # Obtener el escalador correspondiente a 'predicted_y' para este producto
    scaler = scalers['y'][(product_id)]
    
    # Aplicar la inversa del escalador a la predicción 'predicted_y' para este producto
    inverse_scaled_prediction = scaler.inverse_transform([[scaled_prediction]])[0][0]
    
    # Reemplazar la predicción escalada con la predicción invertida en el DataFrame final
    predictions_df_kaggle.at[index, 'predicted_y'] = inverse_scaled_prediction

In [250]:
prediction_list = []
for product_id, pronostico in predictions_kaggle.items():
    prediction_list.append({'product_id': product_id, 'predicted_y': pronostico})

# Convert the list of dictionaries to a DataFrame
predictions_df = pd.concat([predictions_df_kaggle, pd.DataFrame(prediction_list)], ignore_index=True)

# Save the final DataFrame to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.')
display(predictions_df.shape)

Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.


(780, 2)